# Guía de Despliegue de Modelos ML en Render

Este notebook proporciona una guía completa para desplegar modelos de Machine Learning en **Render** utilizando dos enfoques:

1. **Despliegue sin Docker** (Nativo)
2. **Despliegue con Docker** (Containerizado)

---

## ¿Qué es Render?

Render es una plataforma de cloud computing moderna que permite desplegar aplicaciones web, APIs, bases de datos y más de forma sencilla. Es una alternativa a Heroku con un plan gratuito más generoso.

**Características principales:**
- ✅ Plan gratuito disponible
- ✅ Integración con GitHub/GitLab
- ✅ Despliegue automático (CI/CD)
- ✅ Soporte para Docker
- ✅ SSL gratuito
- ✅ Variables de entorno

---

## Requisitos Previos

1. **Cuenta en Render**: Crear una cuenta gratuita en [render.com](https://render.com)
2. **Repositorio Git**: Tu código debe estar en GitHub o GitLab
3. **Aplicación Flask/FastAPI**: Una API funcional para tu modelo
4. **Modelo entrenado**: Tu modelo ML guardado (pickle, joblib, h5, etc.)

---

# PARTE 1: Despliegue SIN Docker

## 📋 Estructura del Proyecto

```
mi-proyecto-ml/
│
├── app.py                  # Aplicación Flask/FastAPI
├── requirements.txt        # Dependencias de Python
├── model.pkl              # Modelo entrenado
├── scaler.pkl             # Preprocesadores (opcional)
└── README.md              # Documentación
```

## 1.1 Crear la Aplicación Flask

Ejemplo de una API sencilla con Flask:

In [ ]:
# app.py - Ejemplo de API con Flask

from flask import Flask, request, jsonify
import pickle
import numpy as np
import os

app = Flask(__name__)

# Cargar el modelo
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Opcional: Cargar scaler u otros preprocesadores
# with open('scaler.pkl', 'rb') as f:
#     scaler = pickle.load(f)

@app.route('/')
def home():
    return jsonify({
        'message': 'API de Machine Learning',
        'version': '1.0',
        'endpoints': ['/predict']
    })

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Obtener datos del request
        data = request.get_json()
        features = np.array(data['features']).reshape(1, -1)
        
        # Realizar predicción
        prediction = model.predict(features)
        
        return jsonify({
            'prediction': prediction.tolist(),
            'status': 'success'
        })
    
    except Exception as e:
        return jsonify({
            'error': str(e),
            'status': 'error'
        }), 400

@app.route('/health')
def health():
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    # Render asigna el puerto dinámicamente
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

## 1.2 Crear requirements.txt

Es **crucial** especificar todas las dependencias:

In [ ]:
# requirements.txt

# Web Framework
flask==2.3.3
gunicorn==21.2.0

# Machine Learning
scikit-learn==1.3.0
numpy==1.24.3
pandas==2.0.3

# Opcional: según tu modelo
# tensorflow==2.13.0
# torch==2.0.1
# xgboost==1.7.6

## 1.3 Configurar Render (Sin Docker)

### Pasos en la interfaz de Render:

1. **Crear un nuevo Web Service**
   - Dashboard → New → Web Service
   - Conectar tu repositorio de GitHub/GitLab

2. **Configuración del servicio:**
   ```
   Name: mi-api-ml
   Environment: Python 3
   Region: Oregon (o el más cercano)
   Branch: main
   Build Command: pip install -r requirements.txt
   Start Command: gunicorn app:app
   ```

3. **Plan:**
   - Free (para pruebas)
   - Starter ($7/mes) o superior (para producción)

4. **Variables de entorno** (opcional):
   - Añadir API keys, configuraciones, etc.

5. **Click en "Create Web Service"**

### ⚠️ Limitaciones del Plan Gratuito:
- El servicio se apaga después de 15 minutos de inactividad
- Primera petición puede tardar ~30 segundos (cold start)
- 750 horas/mes de ejecución

## 1.4 Archivo de Configuración render.yaml (Opcional)

Puedes crear un archivo `render.yaml` en la raíz de tu proyecto para automatizar la configuración:

In [ ]:
# render.yaml

services:
  - type: web
    name: mi-api-ml
    env: python
    buildCommand: pip install -r requirements.txt
    startCommand: gunicorn app:app
    envVars:
      - key: PYTHON_VERSION
        value: 3.11.0
      - key: MODEL_PATH
        value: ./model.pkl

## 1.5 Probar el Despliegue

Una vez desplegado, Render te dará una URL como: `https://mi-api-ml.onrender.com`

Pruébala con Python:

In [ ]:
import requests
import json

# URL de tu API en Render
url = 'https://mi-api-ml.onrender.com/predict'

# Datos de ejemplo
data = {
    'features': [5.1, 3.5, 1.4, 0.2]  # Ejemplo: Iris dataset
}

# Hacer la petición
response = requests.post(url, json=data)

# Mostrar resultado
print('Status Code:', response.status_code)
print('Response:', json.dumps(response.json(), indent=2))

---

# PARTE 2: Despliegue CON Docker

## 📋 Estructura del Proyecto con Docker

```
mi-proyecto-ml/
│
├── app.py                  # Aplicación Flask/FastAPI
├── requirements.txt        # Dependencias de Python
├── Dockerfile             # Instrucciones para crear la imagen
├── .dockerignore          # Archivos a ignorar en la imagen
├── model.pkl              # Modelo entrenado
└── README.md              # Documentación
```

## 2.1 Crear el Dockerfile

El Dockerfile es la receta para construir tu contenedor:

In [ ]:
# Dockerfile

# Imagen base de Python
FROM python:3.11-slim

# Establecer directorio de trabajo
WORKDIR /app

# Copiar archivos de dependencias
COPY requirements.txt .

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Copiar el código de la aplicación
COPY . .

# Exponer el puerto (Render usa la variable PORT)
EXPOSE 5000

# Comando para ejecutar la aplicación
CMD ["gunicorn", "--bind", "0.0.0.0:5000", "app:app"]

## 2.2 Crear .dockerignore

Para evitar copiar archivos innecesarios a la imagen:

In [ ]:
# .dockerignore

# Python
__pycache__
*.pyc
*.pyo
*.pyd
.Python
env/
venv/
.venv

# IDE
.vscode/
.idea/
*.swp
*.swo

# Git
.git
.gitignore

# Documentación
README.md
*.md

# Jupyter
*.ipynb
.ipynb_checkpoints

# Datos de entrenamiento (si son grandes)
data/
*.csv

# Tests
tests/
test_*.py

## 2.3 Probar Docker Localmente

Antes de desplegar, prueba el contenedor en tu máquina:

In [ ]:
# En la terminal:

# 1. Construir la imagen
# docker build -t mi-api-ml:latest .

# 2. Ejecutar el contenedor
# docker run -p 5000:5000 mi-api-ml:latest

# 3. Probar en otra terminal
# curl http://localhost:5000

# 4. Ver logs
# docker logs <container_id>

# 5. Detener el contenedor
# docker stop <container_id>

## 2.4 Configurar Render (Con Docker)

### Opción A: Desde el Dashboard de Render

1. **Crear un nuevo Web Service**
   - Dashboard → New → Web Service
   - Conectar repositorio

2. **Configuración:**
   ```
   Name: mi-api-ml-docker
   Environment: Docker
   Region: Oregon
   Branch: main
   Dockerfile Path: ./Dockerfile (o ./Dockerfile si está en la raíz)
   ```

3. **No necesitas especificar Build/Start Command** (usa el Dockerfile)

4. **Click en "Create Web Service"**

### Opción B: Con render.yaml

In [ ]:
# render.yaml (para Docker)

services:
  - type: web
    name: mi-api-ml-docker
    env: docker
    dockerfilePath: ./Dockerfile
    plan: free
    healthCheckPath: /health
    envVars:
      - key: MODEL_VERSION
        value: v1.0

## 2.5 Manejo del Puerto en Docker

⚠️ **Importante**: Render asigna el puerto dinámicamente mediante la variable `PORT`.

Asegúrate de que tu aplicación use esta variable:

In [ ]:
# En app.py
import os

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

# O en el CMD del Dockerfile con gunicorn:
# CMD gunicorn --bind 0.0.0.0:$PORT app:app

### Dockerfile con variable PORT dinámica:

In [ ]:
# Dockerfile con PORT dinámico

FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# No especificar EXPOSE ya que el puerto es dinámico

# Usar una variable de entorno para el puerto
ENV PORT=5000

# Ejecutar con el puerto dinámico
CMD gunicorn --bind 0.0.0.0:$PORT --workers 2 app:app

---

# Comparación: Docker vs Sin Docker

| Característica | Sin Docker | Con Docker |
|---------------|------------|------------|
| **Simplicidad** | ✅ Más simple | ⚠️ Requiere conocer Docker |
| **Velocidad de build** | ✅ Rápido | ⚠️ Más lento |
| **Reproducibilidad** | ⚠️ Depende del entorno | ✅ Totalmente reproducible |
| **Portabilidad** | ⚠️ Limitada | ✅ Funciona en cualquier lado |
| **Control del entorno** | ⚠️ Limitado | ✅ Control total |
| **Tamaño** | ✅ Ligero | ⚠️ Imagen más pesada |
| **Dependencias del sistema** | ⚠️ Limitadas | ✅ Instalar lo que necesites |
| **Debugging** | ✅ Más fácil | ⚠️ Más complejo |

## ¿Cuándo usar cada opción?

### Sin Docker:
- ✅ Proyectos simples con pocas dependencias
- ✅ Prototipado rápido
- ✅ Solo Python y librerías estándar
- ✅ Equipo sin experiencia en Docker

### Con Docker:
- ✅ Dependencias complejas (ej: librerías del sistema)
- ✅ Necesitas exactamente la misma versión de Python
- ✅ Múltiples servicios (API + DB + Redis, etc.)
- ✅ Proyecto que se desplegará en múltiples plataformas
- ✅ Producción con alta confiabilidad

---

# Mejores Prácticas

## 1. Salud y Monitoreo

In [ ]:
# Añadir endpoints de salud y métricas

@app.route('/health')
def health():
    return jsonify({
        'status': 'healthy',
        'model_loaded': model is not None,
        'timestamp': datetime.now().isoformat()
    })

@app.route('/info')
def info():
    return jsonify({
        'version': '1.0.0',
        'model_type': 'RandomForest',
        'features': ['feature1', 'feature2', 'feature3'],
        'last_updated': '2024-01-15'
    })

## 2. Gestión de Errores

In [ ]:
# Manejo robusto de errores

from functools import wraps
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def handle_errors(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except ValueError as e:
            logger.error(f'ValueError: {str(e)}')
            return jsonify({'error': 'Invalid input', 'details': str(e)}), 400
        except Exception as e:
            logger.error(f'Unexpected error: {str(e)}')
            return jsonify({'error': 'Internal server error'}), 500
    return decorated_function

@app.route('/predict', methods=['POST'])
@handle_errors
def predict():
    data = request.get_json()
    # ... lógica de predicción

## 3. Validación de Entrada

In [ ]:
# Validar datos de entrada

from pydantic import BaseModel, validator
from typing import List

class PredictionRequest(BaseModel):
    features: List[float]
    
    @validator('features')
    def validate_features(cls, v):
        if len(v) != 4:  # Ejemplo: esperas 4 features
            raise ValueError('Se requieren exactamente 4 características')
        if any(x < 0 for x in v):
            raise ValueError('Las características no pueden ser negativas')
        return v

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = PredictionRequest(**request.get_json())
        # Continuar con la predicción...
    except Exception as e:
        return jsonify({'error': str(e)}), 400

## 4. Variables de Entorno

Nunca pongas credenciales en el código. Usa variables de entorno:

In [ ]:
# Usar variables de entorno

import os
from dotenv import load_dotenv

# Para desarrollo local
load_dotenv()

# Obtener configuraciones
MODEL_PATH = os.getenv('MODEL_PATH', 'model.pkl')
API_KEY = os.getenv('API_KEY', '')
DEBUG = os.getenv('DEBUG', 'False') == 'True'

# En Render, añádelas en: Settings → Environment Variables

---

# Solución de Problemas Comunes

## 1. El servicio no arranca

**Síntomas**: Build exitoso pero no responde

**Soluciones**:
- ✅ Verificar que usas `0.0.0.0` como host
- ✅ Usar la variable `PORT` de entorno
- ✅ Revisar los logs en Render (dashboard)
- ✅ Añadir endpoint `/health` y configurarlo

## 2. Error de memoria

**Síntomas**: `MemoryError` o el servicio se reinicia

**Soluciones**:
- ✅ Plan gratuito tiene 512MB RAM
- ✅ Reducir workers de Gunicorn: `--workers 1`
- ✅ Usar modelos más ligeros
- ✅ Cargar el modelo una sola vez (no en cada request)
- ✅ Actualizar a plan de pago

In [ ]:
# Cargar modelo eficientemente

# ❌ MAL - Carga en cada request
@app.route('/predict', methods=['POST'])
def predict():
    model = pickle.load(open('model.pkl', 'rb'))  # ¡NO!
    # ...

# ✅ BIEN - Carga una sola vez al inicio
model = pickle.load(open('model.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def predict():
    # Usar el modelo ya cargado
    prediction = model.predict(features)

## 3. Build muy lento

**Soluciones**:
- ✅ Usar imagen base slim: `python:3.11-slim`
- ✅ Aprovechar cache de Docker (COPY requirements antes que el código)
- ✅ Usar `.dockerignore` correctamente
- ✅ No instalar paquetes innecesarios

## 4. Cold starts lentos

**Síntomas**: Primera petición tarda mucho

**Soluciones**:
- ✅ Plan gratuito se apaga tras 15 min de inactividad
- ✅ Usar plan de pago (no se apaga)
- ✅ Implementar un "ping" periódico (cron job)
- ✅ Mostrar mensaje de carga al usuario

In [ ]:
# Script para mantener el servicio activo (usar con cron-job.org)

import requests
import time
from datetime import datetime

def ping_service(url):
    try:
        response = requests.get(f'{url}/health', timeout=30)
        print(f'[{datetime.now()}] Ping exitoso: {response.status_code}')
    except Exception as e:
        print(f'[{datetime.now()}] Error en ping: {str(e)}')

# Ejecutar cada 10 minutos
if __name__ == '__main__':
    url = 'https://mi-api-ml.onrender.com'
    while True:
        ping_service(url)
        time.sleep(600)  # 10 minutos

---

# Ejemplo Completo: API de Iris Classification

Aquí un ejemplo completo listo para desplegar:

## Paso 1: Entrenar y guardar el modelo

In [ ]:
# train_model.py

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

# Cargar datos
iris = load_iris()
X, y = iris.data, iris.target

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2%}')

# Guardar modelo
with open('iris_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print('Modelo guardado como iris_model.pkl')

## Paso 2: Crear la API

In [ ]:
# app.py

from flask import Flask, request, jsonify
import pickle
import numpy as np
import os

app = Flask(__name__)

# Cargar modelo al inicio
with open('iris_model.pkl', 'rb') as f:
    model = pickle.load(f)

IRIS_SPECIES = ['setosa', 'versicolor', 'virginica']

@app.route('/')
def home():
    return jsonify({
        'message': 'Iris Classification API',
        'version': '1.0',
        'endpoints': {
            '/': 'Info de la API',
            '/predict': 'POST - Clasificar iris',
            '/health': 'Health check'
        },
        'example': {
            'url': '/predict',
            'method': 'POST',
            'body': {
                'sepal_length': 5.1,
                'sepal_width': 3.5,
                'petal_length': 1.4,
                'petal_width': 0.2
            }
        }
    })

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        
        # Extraer features
        features = np.array([
            data['sepal_length'],
            data['sepal_width'],
            data['petal_length'],
            data['petal_width']
        ]).reshape(1, -1)
        
        # Predicción
        prediction = model.predict(features)[0]
        probabilities = model.predict_proba(features)[0]
        
        return jsonify({
            'prediction': IRIS_SPECIES[prediction],
            'prediction_index': int(prediction),
            'probabilities': {
                species: float(prob)
                for species, prob in zip(IRIS_SPECIES, probabilities)
            },
            'confidence': float(max(probabilities)),
            'status': 'success'
        })
    
    except KeyError as e:
        return jsonify({
            'error': f'Campo requerido faltante: {str(e)}',
            'status': 'error'
        }), 400
    
    except Exception as e:
        return jsonify({
            'error': str(e),
            'status': 'error'
        }), 500

@app.route('/health')
def health():
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=False)

## Paso 3: Requirements

In [ ]:
# requirements.txt

flask==2.3.3
gunicorn==21.2.0
scikit-learn==1.3.0
numpy==1.24.3

## Paso 4: Dockerfile

In [ ]:
# Dockerfile

FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

ENV PORT=5000

CMD gunicorn --bind 0.0.0.0:$PORT --workers 2 app:app

## Paso 5: Probar la API

In [ ]:
# test_api.py

import requests
import json

# URL local o de Render
url = 'http://localhost:5000/predict'
# url = 'https://tu-api.onrender.com/predict'

# Datos de prueba
test_data = {
    'sepal_length': 5.1,
    'sepal_width': 3.5,
    'petal_length': 1.4,
    'petal_width': 0.2
}

# Hacer petición
response = requests.post(url, json=test_data)

# Mostrar resultado
print('Status Code:', response.status_code)
print('Response:')
print(json.dumps(response.json(), indent=2))

---

# Recursos Adicionales

## Documentación Oficial
- [Render Docs](https://render.com/docs)
- [Flask Documentation](https://flask.palletsprojects.com/)
- [Gunicorn](https://gunicorn.org/)
- [Docker Documentation](https://docs.docker.com/)

## Alternativas a Render
- **Railway**: Similar a Render, muy fácil de usar
- **Fly.io**: Excelente rendimiento, free tier generoso
- **Google Cloud Run**: Serverless, pago por uso
- **AWS Lambda + API Gateway**: Para despliegues serverless
- **Azure App Service**: Para entornos empresariales

## Tips Extra

1. **Versiona tus modelos**: Usa nombres con versión (`model_v1.pkl`)
2. **Implementa A/B testing**: Despliega múltiples versiones
3. **Usa caché**: Para predicciones repetidas
4. **Añade rate limiting**: Protege tu API de abuso
5. **Implementa autenticación**: API keys para producción
6. **Monitoriza**: Usa herramientas como Sentry o DataDog
7. **Documenta**: Usa Swagger/OpenAPI para tu API